In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Activation
from pyhht.emd import EMD

high_freq_predict = []
low_freq_predict = []

dataset_train = pd.read_csv('data_train_1.csv')# 5164
dataset_test = pd.read_csv('data_test_1.csv') #574
data_no_standard_train = pd.read_csv('train_set_nostandard_1.csv') #5174
data_no_standard_test = pd.read_csv('test_set_nostandard_1.csv') #574
step = 10

while len(dataset_train) < 5264:
#EMD
    data = pd.DataFrame(data_no_standard_train['predict'],index = data_no_standard_train['date'],columns=['predict'])
    decomposer = EMD(data['predict'])               
    imfs = decomposer.decompose()
    low_freq = np.sum(imfs[3:],axis = 0)[10:]
    #high_freq = np.array(data['predict']-low_freq)
    #dataset_train.loc[:,'high_freq'] = high_freq[10:]
    #high_freq_predict+=(high_freq[-10:].squeeze().tolist())
    #print(high_freq_predict)

#标准化    
    sc_x = MinMaxScaler(feature_range = (0, 1))
    sc_y_low_freq = MinMaxScaler(feature_range = (0, 1))
    
    features = ['T10Y3M','curvature','BAMLCC0A1AAATRIV','DPRIME','VIXCLS','DJIA','SP500','twexb','twexm','twexo','DCPF3M','DCPN3M','DFF','NASDAQCOM','DTWEXB','TEDRATE','USD3MTD156N','USEPUINDXD','WILL5000INDFC','WILLRESIND','volatility10d','ma3d','ma10d']
    training_set_scaled_x = sc_x.fit_transform(dataset_train.loc[:len(dataset_train)-step,features])
    #print(training_set_scaled_x.shape)
    
    y = low_freq[:len(dataset_train)-step+1].reshape(-1,1)
    #print(y.shape)
    for i in range(1,step):
        y = np.hstack((y,low_freq[i:len(dataset_train)-step+i+1].reshape(-1,1)))
    #print(y.shape)
    training_set_scaled_y_low_freq = sc_y_low_freq.fit_transform(y)
    #print(training_set_scaled_y_low_freq[:10])
    
    X_train, y_train_low_freq = training_set_scaled_x, training_set_scaled_y_low_freq
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    #print(y_train_low_freq.shape)

#训练模型，提取低频信息
    u2=100
    d2=0.3
    num_stack_layers2 = 3

    regressor2 = Sequential()

    regressor2.add(LSTM(units = u2, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
    regressor2.add(Dropout(d2))
    for i in range(num_stack_layers2-2):
        regressor2.add(LSTM(units = u2, return_sequences = True))
        regressor2.add(Dropout(d2))
    regressor2.add(LSTM(units = u2))
    regressor2.add(Dropout(d2))

    regressor2.add(Dense(units = y_train_low_freq.shape[1]))

    regressor2.compile(optimizer = 'adam', loss = 'mean_squared_error')

    regressor2.fit(X_train, y_train_low_freq, epochs = 10, batch_size = 32)

#样本外预测高频&低频
    test_set = dataset_test.iloc[:,1:].values
    X_test = np.array(dataset_test.loc[0,features]).reshape(1,-1)

    X_test_scaled = sc_x.transform(X_test)

    X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0],X_test_scaled.shape[1],1))

#     y_predict_scaled_high_freq = regressor1.predict(X_test_scaled)
    y_predict_scaled_low_freq = regressor2.predict(X_test_scaled)
    
#     y_predict_high_freq = sc_y_high_freq.inverse_transform(y_predict_scaled_high_freq)
    y_predict_low_freq = sc_y_low_freq.inverse_transform(y_predict_scaled_low_freq)
    
#     high_freq_predict.append(y_predict_high_freq[0][0])
    low_freq_predict.append(y_predict_low_freq)
    print(low_freq_predict)

#更新数据
    dataset_train.loc[len(dataset_train)] = dataset_test.loc[0]
    data_no_standard_train.loc[len(data_no_standard_train)] = data_no_standard_test.loc[0]
    dataset_test = dataset_test.drop([0]).reset_index(drop = True)
    data_no_standard_test = data_no_standard_test.drop([0]).reset_index(drop = True)
    